In [1]:
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.preprocessing import sequence
import numpy as np
from tqdm import tqdm
from IPython.display import clear_output
import time
tf.enable_eager_execution()
tf.test.is_gpu_available()

import sys
sys.path.append('../')

from config import *

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0" 

In [3]:
### Hyperparameter
###################
SAVE_PATH = '/home/huangzc/competition/tencent/model_ckpt/LSTM/model_all.ckpt'
EMB_SIZE = 50
BATCH_SIZE = 1024
EPOCHS = 100
TARGET = 'age'
COLS_NAME = ['creative_id', 'advertiser_id', 'ad_id', 'product_id', 'product_category', 'industry']

In [4]:
### Read data
### TARGET DF
tr_user_df = pd.read_pickle(TRAIN_DIR+USER_PATH)
tr_user_df = tr_user_df.groupby(['user_id']).agg({'age': 'first', 'gender': 'first'}).reset_index()

In [5]:
tr_ad_df = pd.read_pickle(TRAIN_DIR+AD_PATH)
ts_ad_df = pd.read_pickle(TEST_DIR+AD_PATH)

In [6]:
train_df = []
test_df = []
for col in tqdm(COLS_NAME):
    train_df.append(pd.read_pickle(TRAIN_DIR+CLK_PATH_DICT[col]))
    test_df.append(pd.read_pickle(TRAIN_DIR+CLK_PATH_DICT[col]))
    
train_df.append(tr_user_df)

100%|██████████| 6/6 [03:12<00:00, 32.16s/it]


In [7]:
def get_grid_df(creative_df, advertiser_df, ad_df, product_id_df, product_cat_df, industry_df, user_df=None):
    if user_df is  None:
        user_df = creative_df[['user_id']]
        user_df[TARGET] = np.nan
    assert user_df['user_id'].values.tolist() == creative_df['user_id'].values.tolist() \
    == ad_df['user_id'].values.tolist() == product_id_df['user_id'].values.tolist() \
    == product_cat_df['user_id'].values.tolist() == industry_df['user_id'].values.tolist() \
    == advertiser_df['user_id'].values.tolist()

    del advertiser_df['user_id'], ad_df['user_id'], product_id_df['user_id'], product_cat_df['user_id'], industry_df['user_id']
    
    grid_df = pd.concat([creative_df, advertiser_df, 
                         ad_df, product_id_df, 
                         product_cat_df, industry_df,
                         user_df[[TARGET]]], axis=1)

    return grid_df

In [8]:
grid_df = get_grid_df(*train_df)
grid_df_test = get_grid_df(*test_df)
grid_df[TARGET] = grid_df[TARGET] - 1

In [9]:
train, val = train_test_split(grid_df, test_size=0.2, random_state=2020)
test = grid_df_test

In [10]:
def pad_feature():  
    x_train = []
    x_val = []
    x_test = []
    
    sentence_size = max(int(grid_df[COLS_NAME[0]].map(lambda x: len(x)).quantile(0.6)), 
                        int(grid_df_test[COLS_NAME[0]].map(lambda x: len(x)).quantile(0.6)))
    print('choose sentences max len: %d' % (sentence_size))
    print("Pad sequences (samples x time)")      
    for col in tqdm(COLS_NAME):
        x_train.append(sequence.pad_sequences(train[col],
                                             maxlen=sentence_size, 
                                             padding='post', 
                                             truncating='post',
                                             dtype='int64',
                                             value=0
                                             ))
        x_val.append(sequence.pad_sequences(val[col],
                                             maxlen=sentence_size, 
                                             padding='post', 
                                             truncating='post',
                                             dtype='int64',
                                             value=0
                                             ))
        x_test.append(sequence.pad_sequences(test[col], 
                                            maxlen=sentence_size, 
                                            padding='post',
                                            truncating='post',
                                            dtype='int64',
                                            value=0
                                           ))
    print('feature count: train->%d, valid->%d, test->%d' %(len(x_train), len(x_val), len(x_test)))
    return x_train, x_val, x_test

In [11]:
x_train, x_val, x_test = pad_feature()

  0%|          | 0/6 [00:00<?, ?it/s]

choose sentences max len: 29
Pad sequences (samples x time)


100%|██████████| 6/6 [01:34<00:00, 15.68s/it]

feature count: train->6, valid->6, test->6


In [12]:
### Get tf dataset
def get_train_ds(x, y): 
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    dataset = dataset.shuffle(buffer_size=len(x))
    dataset = dataset.batch(BATCH_SIZE)
#     dataset = dataset.repeat(EPOCHS)
    return dataset

def get_test_ds(x, ): 
    dataset = tf.data.Dataset.from_tensor_slices(x).batch(BATCH_SIZE)
    return dataset

train_ds = get_train_ds(tuple(x_train), train[TARGET].values)
valid_ds = get_train_ds(tuple(x_val), val[TARGET].values)
test_ds = get_test_ds(tuple(x_test))

In [13]:
vocab_sizes = []
temp = pd.concat([tr_ad_df, ts_ad_df], axis=0)
for col in COLS_NAME:
    print(col)
    vocab_sizes.append(max(temp[col].unique().tolist()) + 1) ### padding 0 need add 1

creative_id
advertiser_id
ad_id
product_id
product_category
industry


In [14]:
# weights = np.load('/home/huangzc/competition/tencent/data/train_preliminary/gensim_dict.npy')

In [15]:
### Construct Model
#################################
class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
#         self.embedding = tf.keras.layers.Embedding(vocab_size, EMB_SIZE, weights=[weights])
        self.embeddings = []
        for s in vocab_sizes:
            self.embeddings.append(tf.keras.layers.Embedding(s, EMB_SIZE))
        self.concat = tf.keras.layers.Concatenate(axis=-1)
        
        ### LSTM
        self.lstms = []
        for i in range(1):
            self.lstms.append(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)))
        self.lstms.append(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)))
        
        self.dense1 = tf.keras.layers.Dense(64, activation='relu')
        self.dense2 = tf.keras.layers.Dense(32, activation='relu')
        self.dense3 = tf.keras.layers.Dense(10, activation='softmax')

    def call(self, inputs):
        embs = []
        for emb, inp in zip(self.embeddings, inputs):
            x = emb(inp)            
            embs.append(x)
        x = self.concat(embs)
        
        for lstm in self.lstms:
            x = lstm(x)
 
        x = self.dense1(x)
        x = self.dense2(x)
        return self.dense3(x)

model = MyModel()

In [16]:
# # Restore the weights
# model.load_weights(SAVE_PATH)

In [17]:
### Loss & Metric
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

optimizer = tf.keras.optimizers.Adam()

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

valid_loss = tf.keras.metrics.Mean(name='valid_loss')
valid_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='valid_accuracy')

In [18]:
@tf.function
def train_step(features, labels):
    with tf.GradientTape() as tape:
        predictions = model(features)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)

In [19]:
@tf.function
def valid_step(features, labels):
    predictions = model(features)
    v_loss = loss_object(labels, predictions)

    valid_loss(v_loss)
    valid_accuracy(labels, predictions)

In [20]:
for epoch in range(EPOCHS):
  # 在下一个epoch开始时，重置评估指标
    train_loss.reset_states()
    train_accuracy.reset_states()
    valid_loss.reset_states()
    valid_accuracy.reset_states()

    CNT = 0
    for features, labels in train_ds:
        begin = time.time()
        train_step(features, labels)
        CNT += 1
        print('batch: {:d}, batch train loss: {:.2f}, train acc: {:.2f}%, consuming tine: {:.2f}'.
              format(CNT, train_loss.result(), train_accuracy.result()*100, time.time()-begin))

        if CNT % 50 == 0: 
            for val_features, val_labels in valid_ds:
                valid_step(val_features, val_labels)
            print('##################################################')
            print('batch: {:d}, batch valid loss: {:.2f}, valid acc: {:.2f}%'.format(CNT, valid_loss.result(), valid_accuracy.result()*100)) 
            print('##################################################')
            
    template = 'Epoch {}, Loss: {:.2f}, Accuracy: {:.2f}%, Valid Loss: {:.2f}, Valid Accuracy: {:.2f}%'
    print (template.format(epoch+1,
                         train_loss.result(),
                         train_accuracy.result()*100,
                         valid_loss.result(),
                         valid_accuracy.result()*100))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
batch: 1, batch train loss: 2.30, train acc: 6.54%, consuming tine: 19.06
batch: 2, batch train loss: 2.30, train acc: 9.13%, consuming tine: 7.68
batch: 3, batch train loss: 2.30, train acc: 11.43%, consuming tine: 7.68
batch: 4, batch train loss: 2.29, train acc: 13.45%, consuming tine: 5.72
batch: 5, batch train loss: 2.29, train acc: 14.57%, consuming tine: 5.31
batch: 6, batch train loss: 2.28, train acc: 15.59%, consuming tine: 5.88
batch: 7, batch train loss: 2.27, train acc: 16.43%, consuming tine: 5.07
batch: 8, batch train loss: 2.26, train acc: 17.18%, consuming tine: 4.93
batch: 9, batch train loss: 2.25, train acc: 17.78%, consuming tine: 5.27
batch: 10, batch train loss: 2.24, train acc: 18.22%, consuming tine: 4.89
batch: 11, batch train loss: 2.23, train acc: 18.76%, consuming tine: 5.19
batch: 12, batch train loss: 2.22, train acc: 19.12%, consuming tine: 4.99
batch: 13, batch

batch: 102, batch train loss: 1.93, train acc: 24.83%, consuming tine: 5.52
batch: 103, batch train loss: 1.93, train acc: 24.90%, consuming tine: 5.08
batch: 104, batch train loss: 1.92, train acc: 24.97%, consuming tine: 5.28
batch: 105, batch train loss: 1.92, train acc: 25.03%, consuming tine: 5.27
batch: 106, batch train loss: 1.92, train acc: 25.09%, consuming tine: 5.30
batch: 107, batch train loss: 1.92, train acc: 25.18%, consuming tine: 5.69
batch: 108, batch train loss: 1.92, train acc: 25.24%, consuming tine: 5.09
batch: 109, batch train loss: 1.91, train acc: 25.29%, consuming tine: 5.25
batch: 110, batch train loss: 1.91, train acc: 25.33%, consuming tine: 5.65
batch: 111, batch train loss: 1.91, train acc: 25.40%, consuming tine: 5.07
batch: 112, batch train loss: 1.91, train acc: 25.44%, consuming tine: 5.38
batch: 113, batch train loss: 1.91, train acc: 25.49%, consuming tine: 4.95
batch: 114, batch train loss: 1.90, train acc: 25.54%, consuming tine: 5.24
batch: 115, 

batch: 206, batch train loss: 1.76, train acc: 29.66%, consuming tine: 5.98
batch: 207, batch train loss: 1.76, train acc: 29.69%, consuming tine: 5.49
batch: 208, batch train loss: 1.76, train acc: 29.72%, consuming tine: 5.79
batch: 209, batch train loss: 1.76, train acc: 29.74%, consuming tine: 5.73
batch: 210, batch train loss: 1.76, train acc: 29.77%, consuming tine: 5.75
batch: 211, batch train loss: 1.76, train acc: 29.81%, consuming tine: 5.70
batch: 212, batch train loss: 1.76, train acc: 29.84%, consuming tine: 5.89
batch: 213, batch train loss: 1.76, train acc: 29.87%, consuming tine: 5.88
batch: 214, batch train loss: 1.76, train acc: 29.89%, consuming tine: 5.90
batch: 215, batch train loss: 1.76, train acc: 29.93%, consuming tine: 5.40
batch: 216, batch train loss: 1.76, train acc: 29.96%, consuming tine: 5.58
batch: 217, batch train loss: 1.75, train acc: 29.99%, consuming tine: 5.68
batch: 218, batch train loss: 1.75, train acc: 30.03%, consuming tine: 5.79
batch: 219, 

batch: 310, batch train loss: 1.69, train acc: 32.08%, consuming tine: 5.87
batch: 311, batch train loss: 1.69, train acc: 32.09%, consuming tine: 5.59
batch: 312, batch train loss: 1.69, train acc: 32.11%, consuming tine: 6.05
batch: 313, batch train loss: 1.69, train acc: 32.13%, consuming tine: 5.45
batch: 314, batch train loss: 1.69, train acc: 32.15%, consuming tine: 6.09
batch: 315, batch train loss: 1.69, train acc: 32.17%, consuming tine: 5.59
batch: 316, batch train loss: 1.69, train acc: 32.19%, consuming tine: 5.82
batch: 317, batch train loss: 1.68, train acc: 32.22%, consuming tine: 5.57
batch: 318, batch train loss: 1.68, train acc: 32.23%, consuming tine: 5.68
batch: 319, batch train loss: 1.68, train acc: 32.24%, consuming tine: 5.90
batch: 320, batch train loss: 1.68, train acc: 32.26%, consuming tine: 5.59
batch: 321, batch train loss: 1.68, train acc: 32.28%, consuming tine: 5.88
batch: 322, batch train loss: 1.68, train acc: 32.31%, consuming tine: 5.59
batch: 323, 

batch: 414, batch train loss: 1.64, train acc: 33.69%, consuming tine: 5.77
batch: 415, batch train loss: 1.64, train acc: 33.71%, consuming tine: 5.40
batch: 416, batch train loss: 1.64, train acc: 33.72%, consuming tine: 5.83
batch: 417, batch train loss: 1.64, train acc: 33.73%, consuming tine: 5.57
batch: 418, batch train loss: 1.64, train acc: 33.74%, consuming tine: 5.77
batch: 419, batch train loss: 1.64, train acc: 33.75%, consuming tine: 5.70
batch: 420, batch train loss: 1.64, train acc: 33.76%, consuming tine: 5.68
batch: 421, batch train loss: 1.64, train acc: 33.77%, consuming tine: 5.64
batch: 422, batch train loss: 1.64, train acc: 33.78%, consuming tine: 5.54
batch: 423, batch train loss: 1.64, train acc: 33.79%, consuming tine: 5.60
batch: 424, batch train loss: 1.64, train acc: 33.81%, consuming tine: 5.42
batch: 425, batch train loss: 1.64, train acc: 33.82%, consuming tine: 5.56
batch: 426, batch train loss: 1.64, train acc: 33.83%, consuming tine: 5.68
batch: 427, 

...

batch: 606, batch train loss: 0.02, train acc: 99.20%, consuming tine: 4.87
batch: 607, batch train loss: 0.02, train acc: 99.20%, consuming tine: 4.98
batch: 608, batch train loss: 0.02, train acc: 99.20%, consuming tine: 5.10
batch: 609, batch train loss: 0.02, train acc: 99.20%, consuming tine: 5.38
batch: 610, batch train loss: 0.02, train acc: 99.21%, consuming tine: 4.89
batch: 611, batch train loss: 0.02, train acc: 99.21%, consuming tine: 5.48
batch: 612, batch train loss: 0.02, train acc: 99.21%, consuming tine: 5.09
batch: 613, batch train loss: 0.02, train acc: 99.21%, consuming tine: 5.31
batch: 614, batch train loss: 0.02, train acc: 99.21%, consuming tine: 4.87
batch: 615, batch train loss: 0.02, train acc: 99.21%, consuming tine: 4.99
batch: 616, batch train loss: 0.02, train acc: 99.21%, consuming tine: 5.29
batch: 617, batch train loss: 0.02, train acc: 99.21%, consuming tine: 5.28
batch: 618, batch train loss: 0.02, train acc: 99.21%, consuming tine: 5.19
batch: 619, 

batch: 5, batch train loss: 0.02, train acc: 99.34%, consuming tine: 5.81
batch: 6, batch train loss: 0.02, train acc: 99.35%, consuming tine: 5.19
batch: 7, batch train loss: 0.02, train acc: 99.19%, consuming tine: 5.10
batch: 8, batch train loss: 0.02, train acc: 99.19%, consuming tine: 5.46
batch: 9, batch train loss: 0.03, train acc: 99.16%, consuming tine: 5.10
batch: 10, batch train loss: 0.03, train acc: 99.15%, consuming tine: 5.30
batch: 11, batch train loss: 0.03, train acc: 99.17%, consuming tine: 5.21
batch: 12, batch train loss: 0.03, train acc: 99.15%, consuming tine: 5.27
batch: 13, batch train loss: 0.02, train acc: 99.17%, consuming tine: 5.29
batch: 14, batch train loss: 0.02, train acc: 99.16%, consuming tine: 5.28
batch: 15, batch train loss: 0.02, train acc: 99.18%, consuming tine: 5.28
batch: 16, batch train loss: 0.02, train acc: 99.19%, consuming tine: 5.00
batch: 17, batch train loss: 0.02, train acc: 99.20%, consuming tine: 5.38
batch: 18, batch train loss: 0

batch: 111, batch train loss: 0.02, train acc: 99.28%, consuming tine: 4.79
batch: 112, batch train loss: 0.02, train acc: 99.28%, consuming tine: 4.76
batch: 113, batch train loss: 0.02, train acc: 99.28%, consuming tine: 4.84
batch: 114, batch train loss: 0.02, train acc: 99.28%, consuming tine: 4.75
batch: 115, batch train loss: 0.02, train acc: 99.28%, consuming tine: 4.77
batch: 116, batch train loss: 0.02, train acc: 99.28%, consuming tine: 4.50
batch: 117, batch train loss: 0.02, train acc: 99.28%, consuming tine: 5.06
batch: 118, batch train loss: 0.02, train acc: 99.28%, consuming tine: 5.02
batch: 119, batch train loss: 0.02, train acc: 99.28%, consuming tine: 4.38
batch: 120, batch train loss: 0.02, train acc: 99.28%, consuming tine: 5.09
batch: 121, batch train loss: 0.02, train acc: 99.28%, consuming tine: 4.59
batch: 122, batch train loss: 0.02, train acc: 99.27%, consuming tine: 4.98
batch: 123, batch train loss: 0.02, train acc: 99.28%, consuming tine: 4.49
batch: 124, 

batch: 215, batch train loss: 0.02, train acc: 99.33%, consuming tine: 5.21
batch: 216, batch train loss: 0.02, train acc: 99.33%, consuming tine: 5.17
batch: 217, batch train loss: 0.02, train acc: 99.33%, consuming tine: 5.19
batch: 218, batch train loss: 0.02, train acc: 99.33%, consuming tine: 5.20
batch: 219, batch train loss: 0.02, train acc: 99.32%, consuming tine: 5.40
batch: 220, batch train loss: 0.02, train acc: 99.32%, consuming tine: 4.97
batch: 221, batch train loss: 0.02, train acc: 99.32%, consuming tine: 5.70


KeyboardInterrupt: 